# 🚀 ONE-CLICK TTS SERVER
Просто запусти все ячейки!

In [ ]:
print("📦 Устанавливаем edge-tts...")
!pip install edge-tts flask -q

In [ ]:
import os
import subprocess
import asyncio
import threading
import time
import re
from flask import Flask, request, Response
import edge_tts

print("✅ Модули импортированы")

In [ ]:
os.system('fuser -k 9999/tcp 2>/dev/null || true')
print("✅ Порт 9999 очищен")

In [ ]:
app = Flask(__name__)

@app.after_request
def cors(response):
    response.headers["Access-Control-Allow-Origin"] = "*"
    return response

@app.route("/tts")
def tts():
    text = request.args.get("text", "")
    voice = request.args.get("voice", "us_young")
    
    if not text:
        return "No text", 400
    
    voices = {
        "us_young": "en-US-ChristopherNeural",
        "us_mature": "en-US-EricNeural", 
        "uk_british": "en-GB-RyanNeural",
        "australian": "en-AU-WilliamNeural",
        "indian": "en-IN-PrabhatNeural"
    }
    
    voice_id = voices.get(voice, voices["us_young"])
    
    async def gen():
        communicate = edge_tts.Communicate(text, voice_id)
        chunks = []
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                chunks.append(chunk["data"])
        return b"".join(chunks)
    
    try:
        audio = asyncio.run(gen())
        return Response(audio, mimetype="audio/mpeg")
    except Exception as e:
        return str(e), 500

@app.route("/")
def home():
    return "TTS Server OK"

def run_server():
    app.run(host="0.0.0.0", port=9999, debug=False, threaded=True)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
time.sleep(3)
print("✅ Сервер запущен: http://localhost:9999")

In [ ]:
print("🌐 Устанавливаем Cloudflared...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared
print("✅ Cloudflared установлен")

In [ ]:
print("🔗 Получаем публичный URL...")

def get_url():
    process = subprocess.Popen(
        ['cloudflared', 'tunnel', '--url', 'http://localhost:9999'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    for i in range(25):
        line = process.stderr.readline()
        if line and '.trycloudflare.com' in line:
            matches = re.findall(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
            if matches:
                process.terminate()
                return matches[0]
        time.sleep(1)
    
    process.terminate()
    return None

url = get_url()

if url:
    print(f"\n🎯 ТВОЙ ПУБЛИЧНЫЙ URL: {url}")
    print(f"🔗 Пример: {url}/tts?text=Hello&voice=us_young")
else:
    print("\n⚠️ URL не найден, используй локальный: http://localhost:9999")

print("\n✅ Готово! Сервер работает.")